In [1]:
# note, you need to add parquet to flink installation, follow instruction as per class session
# you must take new Dockerfile on day2 github folder, then build image, then run the cluster

In [2]:
import os
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment

import get_env
env = get_env.get_remote_env()
t_env = StreamTableEnvironment.create(env)
t_env.get_config().set("parallelism.default", "1")

2025-12-02T03:18:30.914661Z main ERROR Reconfiguration failed: No configuration found for '63e31ee' at 'null' in 'null'
2025-12-02T03:18:32.051116Z Thread-3 ERROR Reconfiguration failed: No configuration found for '65cad32d' at 'null' in 'null'


In [3]:
conf = t_env.get_config().get_configuration()
conf.set_string("fs.allowed-fallback-filesystems", "hadoop")
conf.set_string("fs.gs.project.id", "flink-demo-470113")
conf.set_string("fs.gs.auth.service.account.json.keyfile", "/etc/gcp/key.json")

In [4]:
t_env.execute_sql("""
CREATE TEMPORARY TABLE ratings_csv (
  `userId` INT,
  `movieId` INT,
  `rating` DOUBLE,
  `timestamp` BIGINT
) WITH (
  'connector' = 'filesystem',
  'path' = 'gs://gk2-datalake/bronze/ratings/',
  'format' = 'csv',
  'csv.field-delimiter' = ',',
  'csv.ignore-parse-errors' = 'true',
  'csv.allow-comments' = 'false',
  'csv.first-line-as-header' = 'true'
)
""")


# 3) Get the table and print schema
ratings_csv = t_env.from_path("ratings_csv")
print("=== Schema ===")
ratings_csv.print_schema()

# 4) Print the data (bounded filesystem source -> will finish)
print("=== Data ===")

limited = ratings_csv.limit(5)
limited.execute().print()

=== Schema ===
(
  `userId` INT,
  `movieId` INT,
  `rating` DOUBLE,
  `timestamp` BIGINT
)
=== Data ===
+----+-------------+-------------+--------------------------------+----------------------+
| op |      userId |     movieId |                         rating |            timestamp |
+----+-------------+-------------+--------------------------------+----------------------+
| +I |      <NULL> |      <NULL> |                         <NULL> |               <NULL> |
| +I |           1 |           1 |                            4.0 |            964982703 |
| +I |           1 |           3 |                            4.0 |            964981247 |
| +I |           1 |           6 |                            4.0 |            964982224 |
| +I |           1 |          47 |                            5.0 |            964983815 |
+----+-------------+-------------+--------------------------------+----------------------+
5 rows in set


In [16]:
t_env.execute_sql("DROP TABLE IF EXISTS ratings_parquet")

t_env.execute_sql("""
CREATE TABLE ratings_parquet (
  `userId` INT,
  `movieId` INT,
  `rating` DOUBLE,
  `timestamp` BIGINT
) WITH (
  'connector' = 'filesystem',
  'path' = 'gs://gk2-datalake/silver/ratings/',
  'format' = 'parquet'
)
""")


In [17]:
# REMAINER, flink-parquet-1.20.2.jar , parquet-hadoop-bundle-*.jar needed refer day2 Dockerfile

# clean data, userId should not be null, and rating should be above 0.5

t_env.execute_sql("""
INSERT INTO ratings_parquet
SELECT userId, movieId, rating, `timestamp`
FROM ratings_csv
WHERE userId IS NOT NULL AND rating > 0.5
""")

In [18]:
df = t_env.sql_query("""
    SELECT * FROM ratings_parquet LIMIT 5
""").to_pandas()

print(df)

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
